* [Bonston Housing](https://github.com/phamdinhkhanh/datasets/blob/master/BostonHousing.csv) about predict Bonston home price.

1. Describe statistics & plot for continuous and category variables. Evaluating for distribution properties of each var.

2. Completed 1 pipeline to processing raw data to useful data.

3. Spilit train & test set and choose metric for problem.

4. Choose model, cross-validation to training and evaluating in training set.

5. Re do ex 4 with other models 

6. Plot result of model to choose best model.

7. Based on best model, Grid Search in parameter space of it.

# 1. Pipeline


## 1.1. Overview

The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of [ Boston MA](http://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html). The following describes the dataset columns:

* CRIM - per capita crime rate by town
* ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS - proportion of non-retail business acres per town.
* CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
* NOX - nitric oxides concentration (parts per 10 million)
* RM - average number of rooms per dwelling
* AGE - proportion of owner-occupied units built prior to 1940
* DIS - weighted distances to five Boston employment centres
* RAD - index of accessibility to radial highways
* TAX - full-value property-tax rate per \$10,000
* PTRATIO - pupil-teacher ratio by town
* B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT - % lower status of the population
* MEDV - Median value of owner-occupied homes in \$1000's


In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/phamdinhkhanh/datasets/master/BostonHousing.csv")
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [ ]:
df.info()

In [ ]:
df.describe()

* From get-go, two data coulmns show interesting summeries. They are : ZN (proportion of residential land zoned for lots over 25,000 sq.ft.) with 0 for 25th, 50th percentiles. 
* Second, CHAS: Charles River dummy variable (1 if tract bounds river; 0 otherwise) with 0 for 25th, 50th and 75th percentiles. These summeries are understandable as both variables are conditional + categorical variables. 
* First assumption would be that these coulms may not be useful in regression task such as predicting MEDV (Median value of owner-occupied homes).
* Another interesing fact on the dataset is the max value of MEDV. From the original data description, it says: Variable #14 seems to be censored at 50.00 (corresponding to a median price of $50,000). Based on that, values above 50.00 may not help to predict MEDV. 



## 1.2. Visualize continuous variables

In [ ]:
# Box Plot to check outliers
fig, axs = plt.subplots(nrows=2, ncols=7, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in df.items():
    sns.boxplot(y=k, data=df, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
plt.show();

In [ ]:
# Columns like CRIM, ZN, RM, B seems to have outliers -> see the outliers percentage in every column.
for k, v in df.items():
    q1 = v.quantile(0.25)
    q3 = v.quantile(0.75)
    irq = q3 - q1
    v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
    perc = np.shape(v_col)[0] * 100.0 / np.shape(df)[0]
    print("Column %s outliers = %.2f%%" % (k, perc))

In [ ]:
# Remove MEDV outliers (MEDV = 50.0) before plotting more distributions
df = df[~(df['medv'] >= 50.0)]
print(np.shape(df))

In [ ]:
# Destiny
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

fig, axs = plt.subplots(nrows = 2, ncols = 7, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in df.items():
    sns.distplot(v, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
plt.show();

The histogram also shows that columns CRIM, ZN, B has highly skewed distributions. Also MEDV looks to have a normal distribution (the predictions) and other colums seem to have norma or bimodel ditribution of data except CHAS (which is a discrete variable).



In [ ]:
# Plot the pairwise correlation on data ≈ Heatmapb
plt.figure(figsize=(20, 10))
sns.heatmap(df.corr().abs(),  annot=True)
plt.show();

From correlation matrix, we see TAX and RAD are highly correlated features. The columns LSTAT, INDUS, RM, TAX, NOX, PTRAIO has a correlation score above 0.5 with MEDV which is a good indication of using as predictors. Let's plot these columns against MEDV

In [ ]:
# Scatter (preprocessing remove outliers)
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

# Remove outliers x 
column_sels = ['lstat', 'indus', 'nox', 'ptratio', 'rm', 'tax', 'dis', 'age']
x = df.loc[:,column_sels]
y = df['medv']
x = pd.DataFrame(min_max_scaler.fit_transform(x), columns=column_sels)

# Plot
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for i, k in enumerate(column_sels):
    sns.regplot(y=y, x=x[k], ax=axs[i])
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
plt.show();

So with these analsis, we may try predict MEDV with 'LSTAT', 'INDUS', 'NOX', 'PTRATIO', 'RM', 'TAX', 'DIS', 'AGE' features

## 1.3. Spilit Train & Test Sets

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state= 42)

X_train = df_train.copy()
y_train = df_train.pop("medv")  # Return item and drop from frame.
X_test = df_test.copy()
y_test = df_test.pop("medv")

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

## 1.4. Preprocessing Data

In [ ]:
num_cols = df.columns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

num_pl = Pipeline(
    steps= [("imputer", KNNImputer(n_neighbors = 7)),
    ("scaler", MinMaxScaler())
    ]
)

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[(('num', num_pl, num_cols))]
)

## 1.5. Full Pipeline

In [ ]:
from sklearn.linear_model import Ridge

# Merge pipeline processor with model to full pipeline
completed_pl = Pipeline(
    steps = [("preprocessor", preprocessor),
    ("ridge", Ridge())]
)

# Training 
completed_pl.fit(X_train, y_train)

# Accuracy 
print(completed_pl.score(X_train, y_train), completed_pl.score(X_test, y_test), sep="\n")

# 2. Cross-Validation
Predict Problem: Accuracy ≈ MSE > KFold > Check Pipeline for more algorithms 

## 1. MSE

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

# Formula to calcute score
metric = make_scorer(mean_squared_error)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# K Fold
cv = KFold(n_splits= 40)

# Accuracy
scores = cross_val_score(completed_pl, X_train, y_train, scoring = metric, cv = cv, n_jobs = -1)
scores.mean()   # Must calcute mean score

## 2. Other Models

In [ ]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
# List model chose
models = [Ridge(), SVR(), DecisionTreeRegressor(), KNeighborsRegressor(), GradientBoostingRegressor()]

# Identifying KFold
cv = KFold(n_splits=40)

all_scores = []

# Accracy
for model in models:
  completed_pl = Pipeline(
    steps=[("preprocessor", preprocessor), 
    ('classifier', model)])

  scores = cross_val_score(completed_pl, X_train, y_train, scoring=metric, cv=cv, n_jobs=-1)
  all_scores.append(scores)

for i in range(len(all_scores)):
  print(np.mean(all_scores[i]))

In [ ]:
# Draw boxplot 
model_names = ['Ridge', 'SVR', 'DTR', "KNR",'GBR']
plt.boxplot(all_scores)
plt.xticks(np.arange(len(model_names))+1, model_names, rotation=45, fontsize=16)
plt.show();